### Pre-defined Method

In [2]:
import json
from openai import OpenAI
import pandas as pd
from tqdm import tqdm
import re

In [3]:
import re

SPECIALS = "".join([',', '.']) 

SPECIALS = re.escape(SPECIALS)

def preprocess(text: str):
    text = re.sub(f"[^0-9{SPECIALS}]", "", text)  # f-문자열을 적용시켜 SPECIALS를 넣어줌.
    # 연속 공백 제거
    text = re.sub(" +", " ", text)
    # 좌우 불필요한 공백 제거
    return text.strip()

In [4]:
from sklearn.metrics import mean_absolute_error
import numpy as np

def test_compute_metrics(label, pred):
    MAE = []
    
    for i in range(5):
        if pred[i][-1] == '.':
            pred[i] = pred[i][:-1]
        
        MAE.append(1 - mean_absolute_error([float(label[i])], [float(pred[i])]))
    
    return MAE

In [ ]:
"""
You are an expert in analyzing sentences for the OCEAN personality traits. \
        The OCEAN personality traits assess the following: O (Openness), C (Conscientiousness), E (Extraversion), A (Agreeableness), and N (Neuroticism). \
        Read the given sentence and analyze the extent to which each of the OCEAN personality traits is exhibited in the sentence. \
        Assign a score between 0 and 1 for each trait based on how strongly the trait is represented in the sentence. \
        A score close to 0 means the trait is not represented at all, while a score close to 1 means the trait is very strongly represented. \
        You should provide the output as the scores for the five OCEAN personality traits for the given sentence, similar to the example below. \
        Example: [O: 0.6722, C: 0.8322, E: 0.4860, A: 0.5298, N: 0.9323]
"""

### Zero-shot LLM Inference

In [ ]:
key_path = "/home/visi2/gpt3.5_key.json" 

with open(key_path, 'r') as f:
    key_json = json.load(f)
    openai_client = OpenAI(api_key=key_json['key'])    


FI_test = pd.read_csv("/data/visi2/FI/KETI_preprocessed_labeled_dropdup+seperate+2senconcat_ver0.4.csv")
result_df = pd.DataFrame(columns=["Text", "Inference"])

for data in tqdm(FI_test['transcription'].to_list()):
    user = data
    messages = [
        {
        "role": "system",
        "content":"You are an expert in analyzing sentences for the OCEAN personality traits. \
        The OCEAN personality traits assess the following: O (Openness), C (Conscientiousness), E (Extraversion), A (Agreeableness), and N (Neuroticism). \
        Read the given sentence and analyze the extent to which each of the OCEAN personality traits is exhibited in the sentence. \
        Assign a score between 0 and 1 for each trait based on how strongly the trait is represented in the sentence. \
        A score close to 0 means the trait is not represented at all, while a score close to 1 means the trait is very strongly represented. \
        You should provide the output as the scores for the five OCEAN personality traits for the given sentence, similar to the example below. \
        Example: [O: 0.6722, C: 0.8322, E: 0.4860, A: 0.5298, N: 0.9323] \n",
        },
        {
        "role": "user",
        "content": 'Given Sentence: "' + user + '",\n Results- [O:'
        }
    ]
    
    response = openai_client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0.9,
            max_tokens=1000,
    )
    
    out_text = response.choices[0].message.content.strip()
    tmp_df = pd.DataFrame({'Text': [user], 'Inference': [out_text]})
    result_df = pd.concat([result_df, tmp_df])

In [ ]:
result_df['Inference'].to_list()

In [ ]:
score = []
p = re.compile('0[.]\d*')

for content in result_df['Inference'].to_list():
    tmp = p.findall(content)
    score.append(tmp)

In [ ]:
score

In [ ]:
FI_test = pd.read_csv("/data/visi2/FI/KETI_preprocessed_labeled_dropdup+seperate+2senconcat_ver0.4.csv",
                     index_col=0)
FI_test['label'] = FI_test['OCEAN'].map(lambda x : [float(i) for i in x[1:-1].split(',')])

In [ ]:
MAE_arr = []
error_idx = []

for i in range(len(FI_test)):
    try:
        MAE_arr.append(test_compute_metrics(FI_test['label'].iloc[i][:5], score[i][:5]))
    except Exception as e:
        print(i, e)
        error_idx.append(i)

In [ ]:
error_idx

In [ ]:
re_df = pd.DataFrame(columns=["Text", "Inference"])

for data in tqdm(FI_test['transcription'][error_idx].to_list()):
    user = data
    messages = [
        {
        "role": "system",
        "content":"You are an expert in analyzing sentences for the OCEAN personality traits. \
        The OCEAN personality traits assess the following: O (Openness), C (Conscientiousness), E (Extraversion), A (Agreeableness), and N (Neuroticism). \
        Read the given sentence and analyze the extent to which each of the OCEAN personality traits is exhibited in the sentence. \
        Assign a score between 0 and 1 for each trait based on how strongly the trait is represented in the sentence. \
        A score close to 0 means the trait is not represented at all, while a score close to 1 means the trait is very strongly represented. \
        You should provide the output as the scores for the five OCEAN personality traits for the given sentence, similar to the example below. \
        Example: [O: 0.6722, C: 0.8322, E: 0.4860, A: 0.5298, N: 0.9323]",
        },
        {
        "role": "user",
        "content": 'Given Sentence: "' + user + '",\n Results- [O:'
        }
    ]
    
    response = openai_client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0.9,
            max_tokens=1000,
    )
    out_text = response.choices[0].message.content.strip()
    tmp_df = pd.DataFrame({'Text': [user], 'Inference': [out_text]})
    re_df = pd.concat([re_df, tmp_df])

In [ ]:
re_score = []
p = re.compile('0[.]\d*')

for content in re_df['Inference'].to_list():
    tmp = p.findall(content)
    re_score.append(tmp)

In [ ]:
re_score

In [ ]:
for i, idx in enumerate(error_idx):
    score[idx] = re_score[i]

In [ ]:
MAE_arr = []
error_idx = []

for i in range(len(FI_test)):
    try:
        MAE_arr.append(test_compute_metrics(FI_test['label'].iloc[i][:5], score[i][:5]))
    except Exception as e:
        print(i, e)
        error_idx.append(i)

In [ ]:
np.array(MAE_arr).mean(axis=0)

In [ ]:
np.mean([0.77591929, 0.80950776, 0.82529518, 0.82160068, 0.80991879])

--------------------

### One-shot LLM Inference

In [ ]:
key_path = "/home/visi2/gpt3.5_key.json"

with open(key_path, 'r') as f:
    key_json = json.load(f)
    openai_client = OpenAI(api_key=key_json['key'])    


FI_test = pd.read_csv("/data/visi2/FI/KETI_preprocessed_labeled_dropdup+seperate+2senconcat_ver0.4.csv")
result_df = pd.DataFrame(columns=["Text", "Inference"])

one_shot_text = "상관없죠 아까 특산물 얘기하셔가지고 그거랑 연계하기도 쉬워 보이고"
for data in tqdm(FI_test['transcription'].to_list()):
    user = data
    messages = [
        {
        "role": "system",
        "content":"You are an expert in analyzing sentences for the OCEAN personality traits. \
        The OCEAN personality traits assess the following: O (Openness), C (Conscientiousness), E (Extraversion), A (Agreeableness), and N (Neuroticism). \
        Read the given sentence and analyze the extent to which each of the OCEAN personality traits is exhibited in the sentence. \
        Assign a score between 0 and 1 for each trait based on how strongly the trait is represented in the sentence. \
        A score close to 0 means the trait is not represented at all, while a score close to 1 means the trait is very strongly represented. \
        You should provide the output as the scores for the five OCEAN personality traits for the given sentence, similar to the example below. \
        Example: [O: 0.6722, C: 0.8322, E: 0.4860, A: 0.5298, N: 0.9323] \n",
        },
        {
                "role": "assistant",
                "content": 'Given Sentence: "' + one_shot_text + '",\n Results- [O: 0.7708, C: 0.5416, E: 0.7708, A: 0.8125, N: 0.5313] \n'
            },
        {
        "role": "user",
        "content": 'Given Sentence: "' + user + '",\n Results- [O:'
        }
    ]
    
    response = openai_client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0.9,
            max_tokens=1000,
    )
    out_text = response.choices[0].message.content.strip()
    tmp_df = pd.DataFrame({'Text': [user], 'Inference': [out_text]})
    result_df = pd.concat([result_df, tmp_df])

In [ ]:
result_df['Inference'].to_list()

In [ ]:
score = []
p = re.compile('0[.]\d*')

for content in result_df['Inference'].to_list():
    tmp = p.findall(content)
    score.append(tmp)

In [ ]:
score

In [ ]:
FI_test = pd.read_csv("/data/visi2/FI/KETI_preprocessed_labeled_dropdup+seperate+2senconcat_ver0.4.csv",
                     index_col=0)
FI_test['label'] = FI_test['OCEAN'].map(lambda x : [float(i) for i in x[1:-1].split(',')])

In [ ]:
MAE_arr = []
error_idx = []

for i in range(len(FI_test)):
    try:
        MAE_arr.append(test_compute_metrics(FI_test['label'].iloc[i][:5], score[i][:5]))
    except Exception as e:
        print(i, e)
        error_idx.append(i)

In [ ]:
error_idx

In [ ]:
re_df = pd.DataFrame(columns=["Text", "Inference"])

for data in tqdm(FI_test['transcription'][error_idx].to_list()):
    user = data
    messages = [
        {
        "role": "system",
        "content":"You are an expert in analyzing sentences for the OCEAN personality traits. \
        The OCEAN personality traits assess the following: O (Openness), C (Conscientiousness), E (Extraversion), A (Agreeableness), and N (Neuroticism). \
        Read the given sentence and analyze the extent to which each of the OCEAN personality traits is exhibited in the sentence. \
        Assign a score between 0 and 1 for each trait based on how strongly the trait is represented in the sentence. \
        A score close to 0 means the trait is not represented at all, while a score close to 1 means the trait is very strongly represented. \
        You should provide the output as the scores for the five OCEAN personality traits for the given sentence, similar to the example below. \
        Example: [O: 0.6722, C: 0.8322, E: 0.4860, A: 0.5298, N: 0.9323]",
        },
        {
        "role": "user",
        "content": 'Given Sentence: "' + user + '",\n Results- [O:'
        }
    ]
    
    response = openai_client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0.9,
            max_tokens=1000,
    )
    out_text = response.choices[0].message.content.strip()
    tmp_df = pd.DataFrame({'Text': [user], 'Inference': [out_text]})
    re_df = pd.concat([re_df, tmp_df])

In [ ]:
re_df

In [ ]:
re_score = []
p = re.compile('0[.]\d*')

for content in re_df['Inference'].to_list():
    tmp = p.findall(content)
    re_score.append(tmp)

In [ ]:
re_score

In [ ]:
for i, idx in enumerate(error_idx):
    score[idx] = re_score[i]

In [ ]:
MAE_arr = []
error_idx = []

for i in range(len(FI_test)):
    try:
        MAE_arr.append(test_compute_metrics(FI_test['label'].iloc[i][:5], score[i][:5]))
    except Exception as e:
        print(i, e)
        error_idx.append(i)

In [ ]:
np.array(MAE_arr).mean(axis=0)

In [ ]:
np.mean([0.84711233, 0.81414687, 0.83802557, 0.83684268, 0.8280393])

--------------------

### Five-Shot Inference

In [ ]:
key_path = "/home/visi2/gpt3.5_key.json"

with open(key_path, 'r') as f:
    key_json = json.load(f)
    openai_client = OpenAI(api_key=key_json['key'])    


FI_test = pd.read_csv("/data/visi2/FI/KETI_preprocessed_labeled_dropdup+seperate+2senconcat_ver0.4.csv")
result_df = pd.DataFrame(columns=["Text", "Inference"])

few_shot_text0 = "좀 쌀쌀한 네 이거 처음 보는 거라 좀 봐야 될 것 같아요"
few_shot_text1 = "맥주가 최고긴 해 회전율을 무시할 수 없어 주류는 나와야죠."
few_shot_text2 = "대추 씨 발라 먹어야 하는 거죠? 그쵸"
few_shot_text3 = "수제로 만든 거 있잖아요. 저는 플리마켓 구경하는 거 진짜 좋아하거든요."
few_shot_text4 = "제주도에서 할 수도 있어요 너무 웃겨"

for data in tqdm(FI_test['transcription'].to_list()):
    user = data
    messages = [
        {
        "role": "system",
        "content":"You are an expert in analyzing sentences for the OCEAN personality traits. \
        The OCEAN personality traits assess the following: O (Openness), C (Conscientiousness), E (Extraversion), A (Agreeableness), and N (Neuroticism). \
        Read the given sentence and analyze the extent to which each of the OCEAN personality traits is exhibited in the sentence. \
        Assign a score between 0 and 1 for each trait based on how strongly the trait is represented in the sentence. \
        A score close to 0 means the trait is not represented at all, while a score close to 1 means the trait is very strongly represented. \
        You should provide the output as the scores for the five OCEAN personality traits for the given sentence, similar to the example below. \
        Example: [O: 0.6722, C: 0.8322, E: 0.4860, A: 0.5298, N: 0.9323] \n",
        },
        {
                "role": "assistant",
                "content": 'Given Sentence: "' + few_shot_text0 + '",\n Results- [O: 0,7708, C: 0.5417, E: 0.7708, A: 0.8125, N: 0.3542] \n'
            },
       {
                "role": "assistant",
                "content": 'Given Sentence: "' + few_shot_text1 + '",\n Results- [O: 0.5833, C: 0.5625, E: 0.4792, A: 0.4583, N: 0.3125] \n'
            },
        {
                "role": "assistant",
                "content": 'Given Sentence: "' + few_shot_text2 + '",\n Results- [O: 0.6875, C: 0.4792, E: 0.4375, A: 0.7708, N: 0.4375] \n'
            },
        {
                "role": "assistant",
                "content": 'Given Sentence: "' + few_shot_text3 + '",\n Results- [O: 0.7292, C: 0.4792, E: 0.5, A: 0.4583, N: 0.7292] \n'
            },
        {
                "role": "assistant",
                "content": 'Given Sentence: "' + few_shot_text4 + '",\n Results- [O: 0.5208, C: 0.6875, E: 0.7917, A: 0.6458, N: 0.25] \n'
            },
        {
        "role": "user",
        "content": 'Given Sentence: "' + user + '",\n Results- [O:'
        }
    ]

    response = openai_client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0.9,
            max_tokens=1000,
    )
    out_text = response.choices[0].message.content.strip()
    tmp_df = pd.DataFrame({'Text': [user], 'Inference': [out_text]})
    result_df = pd.concat([result_df, tmp_df])

In [ ]:
result_df['Inference'].to_list()

In [ ]:
score = []
p = re.compile('0[.]\d*')

for content in result_df['Inference'].to_list():
    tmp = p.findall(content)
    score.append(tmp)

In [ ]:
score

In [ ]:
FI_test = pd.read_csv("/data/visi2/FI/KETI_preprocessed_labeled_dropdup+seperate+2senconcat_ver0.4.csv",
                     index_col=0)
FI_test['label'] = FI_test['OCEAN'].map(lambda x : [float(i) for i in x[1:-1].split(',')])

In [ ]:
MAE_arr = []
error_idx = []

for i in range(len(FI_test)):
    try:
        MAE_arr.append(test_compute_metrics(FI_test['label'].iloc[i][:5], score[i][:5]))
    except Exception as e:
        print(i, e)
        error_idx.append(i)

In [ ]:
error_idx

In [ ]:
re_df = pd.DataFrame(columns=["Text", "Inference"])

for data in tqdm(FI_test['transcription'][error_idx].to_list()):
    user = data
    messages = [
        {
        "role": "system",
        "content":"You are an expert in analyzing sentences for the OCEAN personality traits. \
        The OCEAN personality traits assess the following: O (Openness), C (Conscientiousness), E (Extraversion), A (Agreeableness), and N (Neuroticism). \
        Read the given sentence and analyze the extent to which each of the OCEAN personality traits is exhibited in the sentence. \
        Assign a score between 0 and 1 for each trait based on how strongly the trait is represented in the sentence. \
        A score close to 0 means the trait is not represented at all, while a score close to 1 means the trait is very strongly represented. \
        You should provide the output as the scores for the five OCEAN personality traits for the given sentence, similar to the example below. \
        Example: [O: 0.6722, C: 0.8322, E: 0.4860, A: 0.5298, N: 0.9323]",
        },
        {
        "role": "user",
        "content": 'Given Sentence: "' + user + '",\n Results- [O:'
        }
    ]
    
    response = openai_client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0.9,
            max_tokens=1000,
    )
    out_text = response.choices[0].message.content.strip()
    tmp_df = pd.DataFrame({'Text': [user], 'Inference': [out_text]})
    re_df = pd.concat([re_df, tmp_df])

In [ ]:
re_df

In [ ]:
re_score = []
p = re.compile('0[.]\d*')

for content in re_df['Inference'].to_list():
    tmp = p.findall(content)
    re_score.append(tmp)

In [ ]:
re_score

In [ ]:
for i, idx in enumerate(error_idx):
    score[idx] = re_score[i]

In [ ]:
MAE_arr = []
error_idx = []

for i in range(len(FI_test)):
    try:
        MAE_arr.append(test_compute_metrics(FI_test['label'].iloc[i][:5], score[i][:5]))
    except Exception as e:
        print(i, e)
        error_idx.append(i)

In [ ]:
np.array(MAE_arr).mean(axis=0)

In [ ]:
np.mean([0.84713375, 0.81421225, 0.83817587, 0.83683208, 0.82817295])

### Ten Shot

In [ ]:
key_path = "/home/visi2/gpt3.5_key.json"

with open(key_path, 'r') as f:
    key_json = json.load(f)
    openai_client = OpenAI(api_key=key_json['key'])    


FI_test = pd.read_csv("/data/visi2/FI/KETI_preprocessed_labeled_dropdup+seperate+2senconcat_ver0.4.csv")
result_df = pd.DataFrame(columns=["Text", "Inference"])

few_shot_text0 = "너무 좋다 너무 좋다 롯데월드 기준이면 한강 한강을 여의도로 가는 게 쇼핑 때문이었잖아요. \n"
few_shot_text1 = "그냥 우리집 근처에서 잡자 우리집 서울 하고 서울에서 이제 처음에 이제 먹어야 되잖아요. \
바로 먹을까요 아니면 이게 시장이니까 시장 어 이거 막 뭐 팔잖아요 작게 간단하게 거기서 요기거리나 먹으면서 문화로 가도 상관없을 것 같기도 하고 \n"
few_shot_text2 = "티켓팅 어떻게 성공하는 걸로 하자 \n"
few_shot_text3 = "그러니까 약간 뮤지컬이나 이런 업계 소수 분야로 봤을 때는 탑인데 이제 tv나 이런 약간 저희가 비춰졌을 때 맨날 노출되는 분은 아니다 보니 아무래도 그러시죠 \n"
few_shot_text4 = "그게 또 이게 외국에서 또 뭐 흔할 수도 있을거 같아서 뭐가 있지? 서울에 뭐가 있지 또 \n"
few_shot_text5 = "일단은 저는 저도 비슷한 것 같아요.  근데 일단은 참여하는 사람들이 20대 초반 초중반이랑 40대 중반부터 50대 중반까지잖아요. \n"
few_shot_text6 = "술 안 좋아하시죠? 맥주 좋아하세 \n"
few_shot_text7 = "저도 이수근은 4등입니다.  그래서 나영석 아이유 이종석 이수근이어가지고 \n"
few_shot_text8 = "심리 치료 상담 이런 걸 하시고 요새 요새 티비에 좀 많이 나오긴 했는데 \n"
few_shot_text9 = "전 다 된 것 같습니다. 아까처럼 \n"

for data in tqdm(FI_test['transcription'].to_list()):
    user = data
    messages = [
        {
        "role": "system",
        "content":"You are an expert in analyzing sentences for the OCEAN personality traits. \
        The OCEAN personality traits assess the following: O (Openness), C (Conscientiousness), E (Extraversion), A (Agreeableness), and N (Neuroticism). \
        Read the given sentence and analyze the extent to which each of the OCEAN personality traits is exhibited in the sentence. \
        Assign a score between 0 and 1 for each trait based on how strongly the trait is represented in the sentence. \
        A score close to 0 means the trait is not represented at all, while a score close to 1 means the trait is very strongly represented. \
        You should provide the output as the scores for the five OCEAN personality traits for the given sentence, similar to the example below. \
        Example: [O: 0.6722, C: 0.8322, E: 0.4860, A: 0.5298, N: 0.9323] \n",
        },
        {
                "role": "assistant",
                "content": 'Given Sentence: "' + few_shot_text0 + '",\n Results- [O: 0.7708, C: 0.7083, E: 0.6667, A: 0.5208, N: 0.5417] \n'
            },
       {
                "role": "assistant",
                "content": 'Given Sentence: "' + few_shot_text1 + '",\n Results- [O: 0.6458, C: 0.5208, E: 0.7917, A: 0.6667, N: 0.2292] \n'
            },
        {
                "role": "assistant",
                "content": 'Given Sentence: "' + few_shot_text2 + '",\n Results- [O: 0.4583, C: 0.6875, E: 0.7083, A: 0.5833, N: 0.3333] \n'
            },
        {
                "role": "assistant",
                "content": 'Given Sentence: "' + few_shot_text3 + '",\n Results- [O: 0.5625, C: 0.5417, E: 0.6667, A: 0.5208, N: 0.4375] \n'
            },
        {
                "role": "assistant",
                "content": 'Given Sentence: "' + few_shot_text4 + '",\n Results- [O: 0.6458, C: 0.5208, E: 0.7917, A: 0.6667, N: 0.2292] \n'
            },
        {
                "role": "assistant",
                "content": 'Given Sentence: "' + few_shot_text5 + '",\n Results- [O: 0.6458, C: 0.6458, E: 0.5208, A: 0.7292, N: 0.6250] \n'
            },
       {
                "role": "assistant",
                "content": 'Given Sentence: "' + few_shot_text6 + '",\n Results- [O: 0.4375, C: 0.7083, E: 0.5625, A: 0.3958, N: 0.4167] \n'
            },
        {
                "role": "assistant",
                "content": 'Given Sentence: "' + few_shot_text7 + '",\n Results- [O: 0.6458, C: 0.6458, E: 0.5208, A: 0.7292, N: 0.6250] \n'
            },
        {
                "role": "assistant",
                "content": 'Given Sentence: "' + few_shot_text8 + '",\n Results- [O: 0.5833, C: 0.6250, E: 0.3542, A: 0.5833, N: 0.6250] \n'
            },
        {
                "role": "assistant",
                "content": 'Given Sentence: "' + few_shot_text9 + '",\n Results- [O: 0.4375, C: 0.7083, E: 0.5625, A: 0.3958, N: 0.4167] \n'
            },
        {
        "role": "user",
        "content": 'Given Sentence: "' + user + '",\n Results- [O:'
        }
    ]

    response = openai_client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0.9,
            max_tokens=1000,
    )
    out_text = response.choices[0].message.content.strip()
    tmp_df = pd.DataFrame({'Text': [user], 'Inference': [out_text]})
    result_df = pd.concat([result_df, tmp_df])

In [ ]:
result_df['Inference'].to_list()

In [ ]:
score = []
p = re.compile('0[.]\d*')

for content in result_df['Inference'].to_list():
    tmp = p.findall(content)
    score.append(tmp)

In [ ]:
score

In [ ]:
FI_test = pd.read_csv("/data/visi2/FI/KETI_preprocessed_labeled_dropdup+seperate+2senconcat_ver0.4.csv",
                     index_col=0)
FI_test['label'] = FI_test['OCEAN'].map(lambda x : [float(i) for i in x[1:-1].split(',')])

In [ ]:
FI_test['label'] = FI_test['label'].str.split(' ')

In [ ]:
MAE_arr = []
error_idx = []

In [ ]:
for i in range(len(FI_test)):
    try:
        MAE_arr.append(test_compute_metrics(FI_test['label'].iloc[i][:5], score[i][:5]))
    except Exception as e:
        print(i, e)
        error_idx.append(i)

In [ ]:
error_idx

In [ ]:
re_df = pd.DataFrame(columns=["Text", "Inference"])

In [ ]:
for data in tqdm(FI_test['transcription'][error_idx].to_list()):
    user = data
    messages = [
        {
        "role": "system",
        "content":"You are an expert in analyzing sentences for the OCEAN personality traits. \
        The OCEAN personality traits assess the following: O (Openness), C (Conscientiousness), E (Extraversion), A (Agreeableness), and N (Neuroticism). \
        Read the given sentence and analyze the extent to which each of the OCEAN personality traits is exhibited in the sentence. \
        Assign a score between 0 and 1 for each trait based on how strongly the trait is represented in the sentence. \
        A score close to 0 means the trait is not represented at all, while a score close to 1 means the trait is very strongly represented. \
        You should provide the output as the scores for the five OCEAN personality traits for the given sentence, similar to the example below. \
        Example: [O: 0.6722, C: 0.8322, E: 0.4860, A: 0.5298, N: 0.9323]",
        },
        {
        "role": "user",
        "content": 'Given Sentence: "' + user + '",\n Results- [O:'
        }
    ]
    
    response = openai_client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0.9,
            max_tokens=1000,
    )
    out_text = response.choices[0].message.content.strip()
    tmp_df = pd.DataFrame({'Text': [user], 'Inference': [out_text]})
    re_df = pd.concat([re_df, tmp_df])


In [ ]:
re_df

In [ ]:
re_score = []
p = re.compile('0[.]\d*')

In [ ]:
for content in re_df['Inference'].to_list():
    tmp = p.findall(content)
    re_score.append(tmp)

In [ ]:
re_score

In [ ]:
for i, idx in enumerate(error_idx):
    score[idx] = re_score[i]

In [ ]:
MAE_arr = []
error_idx = []

In [ ]:
for i in range(len(FI_test)):
    try:
        MAE_arr.append(test_compute_metrics(FI_test['label'].iloc[i][:5], score[i][:5]))
    except Exception as e:
        print(i, e)
        error_idx.append(i)


In [ ]:
np.array(MAE_arr).mean(axis=0)

In [ ]:
np.mean([0.84713375, 0.81421225, 0.83817587, 0.83683208, 0.82817295])

### GPT4 Inference - Zero Shot

In [ ]:
key_path = "/home/visi2/gpt3.5_key.json"

with open(key_path, 'r') as f:
    key_json = json.load(f)
    openai_client = OpenAI(api_key=key_json['key'])    


FI_test = pd.read_csv("/data/visi2/FI/KETI_preprocessed_labeled_dropdup+seperate+2senconcat_ver0.4.csv")
result_df = pd.DataFrame(columns=["Text", "Inference"])

for data in tqdm(FI_test['transcription'].to_list()):
    user = data
    messages = [
        {
        "role": "system",
        "content":"You are an expert in analyzing sentences for the OCEAN personality traits. \
        The OCEAN personality traits assess the following: O (Openness), C (Conscientiousness), E (Extraversion), A (Agreeableness), and N (Neuroticism). \
        Read the given sentence and analyze the extent to which each of the OCEAN personality traits is exhibited in the sentence. \
        Assign a score between 0 and 1 for each trait based on how strongly the trait is represented in the sentence. \
        A score close to 0 means the trait is not represented at all, while a score close to 1 means the trait is very strongly represented. \
        You should provide the output as the scores for the five OCEAN personality traits for the given sentence, similar to the example below. \
        Example: [O: 0.6722, C: 0.8322, E: 0.4860, A: 0.5298, N: 0.9323] \n",
        },
        {
        "role": "user",
        "content": 'Given Sentence: "' + user + '",\n Results- [O:'
        }
    ]
    
    response = openai_client.chat.completions.create(
            model="gpt-4-turbo",
            messages=messages,
            temperature=0.9,
            max_tokens=1000,
    )
    out_text = response.choices[0].message.content.strip()
    tmp_df = pd.DataFrame({'Text': [user], 'Inference': [out_text]})
    result_df = pd.concat([result_df, tmp_df])

In [ ]:
result_df['Inference'].to_list()

In [ ]:
score = []
p = re.compile('0[.]\d*')

for content in result_df['Inference'].to_list():
    tmp = p.findall(content)
    score.append(tmp)

In [ ]:
score

In [ ]:
FI_test = pd.read_csv("/data/visi2/FI/KETI_preprocessed_labeled_dropdup+seperate+2senconcat_ver0.4.csv",
                     index_col=0)
FI_test['label'] = FI_test[['openness', 'conscientiousness', 
                              'extraversion', 'agreeableness', 
                              'neuroticism']].apply(
    lambda row: ' '.join(row.values.astype('str')), axis=1)

FI_test['label'] = FI_test['label'].str.split(' ')

FI_test

In [ ]:
MAE_arr = []
error_idx = []

for i in range(len(FI_test)):
    try:
        MAE_arr.append(test_compute_metrics(FI_test['label'].iloc[i][:5], score[i][:5]))
    except Exception as e:
        print(i, e)
        error_idx.append(i)

In [ ]:
error_idx

In [ ]:
np.array(MAE_arr).mean(axis=0)

In [ ]:
np.mean([0.7756677 , 0.777749  , 0.80246082, 0.81028451, 0.72891629])

### GPT4 Inference - One Shot

In [ ]:
key_path = "/home/visi2/gpt3.5_key.json"

with open(key_path, 'r') as f:
    key_json = json.load(f)
    openai_client = OpenAI(api_key=key_json['key'])    


FI_test = pd.read_csv("/data/visi2/FI/KETI_preprocessed_labeled_dropdup+seperate+2senconcat_ver0.4.csv")
result_df = pd.DataFrame(columns=["Text", "Inference"])

one_shot_text = "현재 가장 좋아하는 책 사실 몇 달 전에 서점에서 이 책을 샀는데 읽을 기회가 없었는데 읽기 시작했어요 한번 시작하면 내려놓고 싶지 않은 책 조이스 마이어의 <용감하게 살기>라는 책이에요."

for data in tqdm(FI_test['transcription'].to_list()):
    user = data
    messages = [
        {
        "role": "system",
        "content":"You are an expert in analyzing sentences for the OCEAN personality traits. \
        The OCEAN personality traits assess the following: O (Openness), C (Conscientiousness), E (Extraversion), A (Agreeableness), and N (Neuroticism). \
        Read the given sentence and analyze the extent to which each of the OCEAN personality traits is exhibited in the sentence. \
        Assign a score between 0 and 1 for each trait based on how strongly the trait is represented in the sentence. \
        A score close to 0 means the trait is not represented at all, while a score close to 1 means the trait is very strongly represented. \
        You should provide the output as the scores for the five OCEAN personality traits for the given sentence, similar to the example below. \
        Example: [O: 0.6722, C: 0.8322, E: 0.4860, A: 0.5298, N: 0.9323] \n",
        },
        {
                "role": "assistant",
                "content": 'Given Sentence: "' + one_shot_text + '",\n Results- [O: 0.6333, C: 0.7184, E: 0.4019, A: 0.4176, N: 0.5313] \n'
            },
        {
        "role": "user",
        "content": 'Given Sentence: "' + user + '",\n Results- [O:'
        }
    ]
    
    response = openai_client.chat.completions.create(
            model="gpt-4-turbo",
            messages=messages,
            temperature=0.9,
            max_tokens=1000,
    )
    out_text = response.choices[0].message.content.strip()
    tmp_df = pd.DataFrame({'Text': [user], 'Inference': [out_text]})
    result_df = pd.concat([result_df, tmp_df])

In [ ]:
result_df['Inference'].to_list()

In [ ]:
score = []
p = re.compile('0[.]\d*')

for content in result_df['Inference'].to_list():
    tmp = p.findall(content)
    score.append(tmp)

In [ ]:
score

In [ ]:
FI_test = pd.read_csv("/data/visi2/FI/KETI_preprocessed_labeled_dropdup+seperate+2senconcat_ver0.4.csv",
                     index_col=0)
FI_test['label'] = FI_test[['openness', 'conscientiousness', 
                              'extraversion', 'agreeableness', 
                              'neuroticism']].apply(
    lambda row: ' '.join(row.values.astype('str')), axis=1)

FI_test['label'] = FI_test['label'].str.split(' ')

FI_test

In [ ]:
MAE_arr = []
error_idx = []

for i in range(len(FI_test)):
    try:
        MAE_arr.append(test_compute_metrics(FI_test['label'].iloc[i][:5], score[i][:5]))
    except Exception as e:
        print(i, e)
        error_idx.append(i)

In [ ]:
error_idx

In [ ]:
np.array(MAE_arr).mean(axis=0)

In [ ]:
np.mean([0.78392659, 0.80709438, 0.82091478, 0.82104526, 0.76704043])

### GPT4 Inference - Five Shot

In [ ]:
key_path = "/home/visi2/gpt3.5_key.json"

with open(key_path, 'r') as f:
    key_json = json.load(f)
    openai_client = OpenAI(api_key=key_json['key'])    


FI_test = pd.read_csv("/data/visi2/FI/KETI_preprocessed_labeled_dropdup+seperate+2senconcat_ver0.4.csv")
result_df = pd.DataFrame(columns=["Text", "Inference"])

few_shot_text0 = "현재 가장 좋아하는 책 사실 몇 달 전에 서점에서 이 책을 샀는데 읽을 기회가 없었는데 읽기 시작했어요 한번 시작하면 내려놓고 싶지 않은 책 조이스 마이어의 <용감하게 살기>라는 책이에요."
few_shot_text1 = "제가 타겟에서 일했던 곳과 월마트에서 일했던 곳 제가 유튜브를하지 않았다면 아마 여전히 그 일을하고 있지 않았을 것입니다. 저는 항상 손으로 일하고 물건을 만드는 데 관심이 있었지만 건설은 그랬어요"
few_shot_text2 = "나쁘지만 그들이 만든 최악의 영화는 아닙니다. 그들이 만든 최악의 영화는 괴물이라고 생각합니다. 괴물은 텍사스의 비료 판매원이 만들었다는 점에서 정말 흥미 롭습니다."
few_shot_text3 = "매우 소외감을 느낀다는 것은 확실히 제 자존감과 제 자신을 보는 방식에 타격을 주었습니다 제가 이것을 극복한 방법은 제가 매우 긍정적인 환경에 있는지 확인했습니다 저는 합창단처럼 제가 좋아하는 일을 계속하고 예술을 만들었습니다."
few_shot_text4 = "감정을 읽는 것에 대해 사람들의 얼굴을 보고 그들의 감정을 어떻게 알 수 있는지에 관한 것입니다 흥미롭습니다 나는 유령 요금소를 읽을 것입니다 나는 그것을 읽지 않았다 나는 많은 사람들이 가지고 있다는 것을 알고 있습니다 미안합니다 나는 그것을 읽지 않았다."


for data in tqdm(FI_test['transcription'].to_list()):
    user = data
    messages = [
        {
        "role": "system",
        "content":"You are an expert in analyzing sentences for the OCEAN personality traits. \
        The OCEAN personality traits assess the following: O (Openness), C (Conscientiousness), E (Extraversion), A (Agreeableness), and N (Neuroticism). \
        Read the given sentence and analyze the extent to which each of the OCEAN personality traits is exhibited in the sentence. \
        Assign a score between 0 and 1 for each trait based on how strongly the trait is represented in the sentence. \
        A score close to 0 means the trait is not represented at all, while a score close to 1 means the trait is very strongly represented. \
        You should provide the output as the scores for the five OCEAN personality traits for the given sentence, similar to the example below. \
        Example: [O: 0.6722, C: 0.8322, E: 0.4860, A: 0.5298, N: 0.9323] \n",
        },
        {
                "role": "assistant",
                "content": 'Given Sentence: "' + few_shot_text0 + '",\n Results- [O: 0.6333, C: 0.7184, E: 0.4019, A: 0.4176, N: 0.5313] \n'
            },
       {
                "role": "assistant",
                "content": 'Given Sentence: "' + few_shot_text1 + '",\n Results- [O: 0.6889, C: 0.5631, E: 0.5981, A: 0.6374, N: 0.6563] \n'
            },
        {
                "role": "assistant",
                "content": 'Given Sentence: "' + few_shot_text2 + '",\n Results- [O: 0.7778, C: 0.4175, E: 0.5794, A: 0.6813, N: 0.65625] \n'
            },
        {
                "role": "assistant",
                "content": 'Given Sentence: "' + few_shot_text3 + '",\n Results- [O: 0.9000, C: 0.8835, E: 0.7103, A: 0.7912, N: 0.8229] \n'
            },
        {
                "role": "assistant",
                "content": 'Given Sentence: "' + few_shot_text4 + '",\n Results- [O: 0.3889, C: 0.4660, E: 0.2056, A: 0.4176, N: 0.3646] \n'
            },
        {
        "role": "user",
        "content": 'Given Sentence: "' + user + '",\n Results- [O:'
        }
    ]

    response = openai_client.chat.completions.create(
            model="gpt-4-turbo",
            messages=messages,
            temperature=0.9,
            max_tokens=1000,
    )
    out_text = response.choices[0].message.content.strip()
    tmp_df = pd.DataFrame({'Text': [user], 'Inference': [out_text]})
    result_df = pd.concat([result_df, tmp_df])

In [ ]:
result_df['Inference'].to_list()

In [ ]:
score = []
p = re.compile('0[.]\d*')

for content in result_df['Inference'].to_list():
    tmp = p.findall(content)
    score.append(tmp)

In [ ]:
score

In [ ]:
FI_test = pd.read_csv("/data/visi2/FI/KETI_preprocessed_labeled_dropdup+seperate+2senconcat_ver0.4.csv",
                     index_col=0)
FI_test['label'] = FI_test[['openness', 'conscientiousness', 
                              'extraversion', 'agreeableness', 
                              'neuroticism']].apply(
    lambda row: ' '.join(row.values.astype('str')), axis=1)

FI_test['label'] = FI_test['label'].str.split(' ')

FI_test

In [ ]:
MAE_arr = []
error_idx = []

for i in range(len(FI_test)):
    try:
        MAE_arr.append(test_compute_metrics(FI_test['label'].iloc[i][:5], score[i][:5]))
    except Exception as e:
        print(i, e)
        error_idx.append(i)

In [ ]:
error_idx

In [ ]:
re_df = pd.DataFrame(columns=["Text", "Inference"])

for data in tqdm(FI_test['transcription'][error_idx].to_list()):
    user = data
    messages = [
        {
        "role": "system",
        "content":"You are an expert in analyzing sentences for the OCEAN personality traits. \
        The OCEAN personality traits assess the following: O (Openness), C (Conscientiousness), E (Extraversion), A (Agreeableness), and N (Neuroticism). \
        Read the given sentence and analyze the extent to which each of the OCEAN personality traits is exhibited in the sentence. \
        Assign a score between 0 and 1 for each trait based on how strongly the trait is represented in the sentence. \
        A score close to 0 means the trait is not represented at all, while a score close to 1 means the trait is very strongly represented. \
        You should provide the output as the scores for the five OCEAN personality traits for the given sentence, similar to the example below. \
        Example: [O: 0.6722, C: 0.8322, E: 0.4860, A: 0.5298, N: 0.9323]",
        },
        {
        "role": "user",
        "content": 'Given Sentence: "' + user + '",\n Results- [O:'
        }
    ]
    
    response = openai_client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0.9,
            max_tokens=1000,
    )
    out_text = response.choices[0].message.content.strip()
    tmp_df = pd.DataFrame({'Text': [user], 'Inference': [out_text]})
    re_df = pd.concat([re_df, tmp_df])

In [ ]:
re_df

In [ ]:
re_score = []
p = re.compile('0[.]\d*')

for content in re_df['Inference'].to_list():
    tmp = p.findall(content)
    re_score.append(tmp)

In [ ]:
re_score

In [ ]:
for i, idx in enumerate(error_idx):
    score[idx] = re_score[i]

In [ ]:
MAE_arr = []
error_idx = []

for i in range(len(FI_test)):
    try:
        MAE_arr.append(test_compute_metrics(FI_test['label'].iloc[i][:5], score[i][:5]))
    except Exception as e:
        print(i, e)
        error_idx.append(i)

In [ ]:
np.array(MAE_arr).mean(axis=0)

In [ ]:
np.mean([0.80206517, 0.82098391, 0.81051178, 0.82531748, 0.75400396])